In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.model_selection import train_test_split
from tqdm import tqdm  # import tqdm for progress bar 
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [2]:
target_labels = ['Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___healthy', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Spot', 'Tomato___Tomato_mosaic_virus', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus']

In [3]:
main_path = 'data_penyakit/dataset/train'

In [4]:
# Load the images and labels
X = []
y = []
for label in target_labels:
    label_dir = os.path.join(main_path, label)
    for img_file in os.listdir(label_dir):
        img_path = os.path.join(label_dir, img_file)
        img = load_img(img_path, target_size=(128, 128))
        img_array = img_to_array(img)
        X.append(img_array)
        y.append(target_labels.index(label))

In [5]:
from tensorflow.keras.utils import to_categorical

In [6]:
X = np.array(X)
y = to_categorical(y, num_classes=len(target_labels))

In [7]:
X.shape

(18345, 128, 128, 3)

In [8]:
y.shape

(18345, 10)

In [9]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
X_train.shape

(14676, 128, 128, 3)

In [11]:
X_test.shape

(3669, 128, 128, 3)

In [12]:
y_train.shape

(14676, 10)

In [13]:
y_test.shape

(3669, 10)

In [14]:
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.models import Sequential

In [15]:
len(target_labels)

10

In [16]:
cnn_model = Sequential()
cnn_model.add(Conv2D(32, kernel_size= (3,3), activation = 'relu',input_shape=(128,128,3)))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(64,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(64,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(96,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(32,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())

cnn_model.add(Dropout(0.2))
cnn_model.add(Flatten())
cnn_model.add(Dense(128, activation = 'relu'))
cnn_model.add(Dropout(0.3))
cnn_model.add(Dense(len(target_labels), activation='softmax'))
cnn_model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [17]:
# Train the CNN model
cnn_history = cnn_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)

Epoch 1/10


2023-05-30 22:20:35.405776: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


459/459 [==============================] - 175s 371ms/step - loss: 1.4141 - accuracy: 0.5085 - val_loss: 1.0910 - val_accuracy: 0.6288
Epoch 2/10
459/459 [==============================] - 145s 315ms/step - loss: 0.7770 - accuracy: 0.7305 - val_loss: 0.5585 - val_accuracy: 0.8065
Epoch 3/10
459/459 [==============================] - 157s 342ms/step - loss: 0.5901 - accuracy: 0.7927 - val_loss: 0.8737 - val_accuracy: 0.6868
Epoch 4/10
459/459 [==============================] - 118s 256ms/step - loss: 0.4698 - accuracy: 0.8351 - val_loss: 0.8329 - val_accuracy: 0.6956
Epoch 5/10
459/459 [==============================] - 122s 266ms/step - loss: 0.4074 - accuracy: 0.8607 - val_loss: 0.5856 - val_accuracy: 0.7795
Epoch 6/10
459/459 [==============================] - 109s 236ms/step - loss: 0.3712 - accuracy: 0.8689 - val_loss: 0.3430 - val_accuracy: 0.8784
Epoch 7/10
459/459 [==============================] - 105s 230ms/step - loss: 0.3216 - accuracy: 0.8917 - val_loss: 0.5470 - val_accura

In [18]:
# Evaluate the performance of the trained CNN model on the test set
cnn_scores = cnn_model.evaluate(X_test, y_test, verbose=0)
print("CNN Model Accuracy: %.2f%%" % (cnn_scores[1] * 100))

CNN Model Accuracy: 89.59%


In [19]:
cnn_scores

[0.2965467572212219, 0.8958844542503357]

In [20]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Use the model to predict the test dataset
y_pred_prob = cnn_model.predict(X_test)

# Convert probabilities to classes
y_pred = np.argmax(y_pred_prob, axis=1)

# If y_test is one-hot encoded, convert it to classes as well
y_test_classes = np.argmax(y_test, axis=1)

# Calculate accuracy
accuracy = accuracy_score(y_test_classes, y_pred)

# Calculate precision
precision = precision_score(y_test_classes, y_pred, average='weighted') 

# Calculate recall
recall = recall_score(y_test_classes, y_pred, average='weighted') 

# Calculate F1 score
f1 = f1_score(y_test_classes, y_pred, average='weighted')

# Print the metrics
print("Accuracy: %.2f%%" % (accuracy * 100))
print("Precision: %.2f%%" % (precision * 100))
print("Recall: %.2f%%" % (recall * 100))
print("F1 score: %.2f%%" % (f1 * 100))

115/115 [==============================] - 6s 51ms/step
Accuracy: 89.59%
Precision: 89.77%
Recall: 89.59%
F1 score: 89.49%


In [21]:
# Extract the features using the trained CNN model
cnn_features = cnn_model.predict(X_train)

459/459 [==============================] - 26s 56ms/step


# rf

In [22]:
import time

In [23]:
# import the Random Forest classifier
from RF_Manual import RandomForest

In [24]:
# Train a Random Forest classifier on the extracted features
start_time = time.time()
rf_classifier = RandomForest(n_trees=100, max_depth=42)
rf_classifier.fit(cnn_features, np.argmax(y_train, axis=1))

# count time for training
end_time = time.time()
training_time = end_time - start_time
print(f"Training time: {training_time} seconds")

Training time: 1254.714162349701 seconds


In [25]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Evaluate the performance of the Random Forest classifier on the test set
rf_features = cnn_model.predict(X_test)
rf_features = rf_features.reshape(rf_features.shape[0], -1)
rf_predictions = rf_classifier.predict(rf_features)
rf_accuracy = np.mean(rf_predictions == np.argmax(y_test, axis=1))
print("Random Forest Classifier Accuracy: %.2f%%" % (rf_accuracy * 100))

# Calculate and print the Precision
rf_precision = precision_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier Precision: %.2f" % rf_precision)

# Calculate and print the Recall
rf_recall = recall_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier Recall: %.2f" % rf_recall)

# Calculate and print the F1 score
rf_f1_score = f1_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier F1 Score: %.2f" % rf_f1_score)

115/115 [==============================] - 13s 109ms/step
Random Forest Classifier Accuracy: 92.61%
Random Forest Classifier Precision: 0.93
Random Forest Classifier Recall: 0.93
Random Forest Classifier F1 Score: 0.93


# tes

In [26]:
#Label assignment
label=['Tomato___Bacterial_spot', 
       'Tomato___Early_blight', 
       'Tomato___healthy', 
       'Tomato___Late_blight', 
       'Tomato___Leaf_Mold', 
       'Tomato___Septoria_leaf_spot', 
       'Tomato___Spider_mites Two-spotted_spider_mite', 
       'Tomato___Target_Spot', 
       'Tomato___Tomato_mosaic_virus', 
       'Tomato___Tomato_Yellow_Leaf_Curl_Virus']

# tes cnn

In [27]:
#path=input("Enter your image path-: ")
def testing(path):
    test_image=load_img(path,target_size=(128,128))
    #print(test_image)
    test_image=img_to_array(test_image)
    test_image=np.expand_dims(test_image,axis=0)
    result = cnn_model.predict(test_image)

    label2=label[result.argmax()]
    print(f"Result is --> {result}")
    print(f"your leaf disease is --> {label2}")
#testing(input("Enter your image path-: "))
path='data_penyakit/tomat/tomato-bacterial-spot-1.JPG'
testing(path)

1/1 [==============================] - 0s 28ms/step
Result is --> [[9.6710968e-01 1.0293041e-03 3.6442669e-03 4.6739902e-04 2.7282338e-04
  1.5571671e-04 6.8867472e-03 2.0046040e-02 5.5563046e-06 3.8255961e-04]]
your leaf disease is --> Tomato___Bacterial_spot


In [28]:
import os

def testing_folder_cnn(path):
    test_image = load_img(path, target_size=(128, 128))
    test_image = img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0)
    result = cnn_model.predict(test_image)

    label2 = label[result.argmax()]
    return label2

folder_path = 'data_penyakit/tomat'

results = []
for filename in os.listdir(folder_path):
    image_path = os.path.join(folder_path, filename)
    try:
        label2 = testing_folder_cnn(image_path)
        print(f"Your leaf disease is --> {label2}")
        results.append({"filename": filename, "disease": label2})  # append result to list
    except IOError:
        print(f"Cannot open {filename}")
    print("\n")

df_cnn = pd.DataFrame(results)
df_cnn.to_csv('results-cnn-tomat.csv', index=False)

1/1 [==============================] - 0s 30ms/step
Your leaf disease is --> Tomato___Septoria_leaf_spot


1/1 [==============================] - 0s 30ms/step
Your leaf disease is --> Tomato___healthy


1/1 [==============================] - 0s 30ms/step
Your leaf disease is --> Tomato___Spider_mites Two-spotted_spider_mite


1/1 [==============================] - 0s 29ms/step
Your leaf disease is --> Tomato___Tomato_Yellow_Leaf_Curl_Virus


1/1 [==============================] - 0s 29ms/step
Your leaf disease is --> Tomato___Bacterial_spot


1/1 [==============================] - 0s 22ms/step
Your leaf disease is --> Tomato___Early_blight


1/1 [==============================] - 0s 55ms/step
Your leaf disease is --> Tomato___Early_blight


1/1 [==============================] - 0s 24ms/step
Your leaf disease is --> Tomato___Bacterial_spot


1/1 [==============================] - 0s 41ms/step
Your leaf disease is --> Tomato___Tomato_Yellow_Leaf_Curl_Virus


1/1 [=======================

# rf

In [29]:
def testing_rf(path, rf_classifier):
    test_image = load_img(path, target_size=(128, 128))
    test_image = img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0)

    rf_features = cnn_model.predict(test_image)
    rf_features = rf_features.reshape(rf_features.shape[0], -1)
    result = rf_classifier.predict(rf_features)

    label2 = label[result[0]]
    print(f"Result is --> {result}")
    print(f"Your leaf disease is --> {label2}")

# testing(input("Enter your image path-: "))
path = 'data_penyakit/tomat/tomato-bacterial-spot-1.JPG'
testing_rf(path, rf_classifier)

1/1 [==============================] - 0s 26ms/step
Result is --> [0]
Your leaf disease is --> Tomato___Bacterial_spot


In [30]:
import os

def testing_folder_rf(path, rf_classifier):
    test_image = load_img(path, target_size=(128, 128))
    test_image = img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0)

    rf_features = cnn_model.predict(test_image)
    rf_features = rf_features.reshape(rf_features.shape[0], -1)
    result = rf_classifier.predict(rf_features)

    label2 = label[result[0]]
    return label2

folder_path = 'data_penyakit/tomat'
results = []

for filename in os.listdir(folder_path):
    image_path = os.path.join(folder_path, filename)
    print(f"Testing image: {filename}")
    try:
        label2 = testing_folder_rf(image_path, rf_classifier)
        print(f"Your leaf disease is --> {label2}")
        results.append({"filename": filename, "disease": label2})  # append result to list
    except IOError:
        print(f"Cannot open {filename}")
    print("\n")

df_rf = pd.DataFrame(results)

# Save the dataframe into a CSV file
df_rf.to_csv('results-rf-tomat.csv', index=False)

Testing image: tomato-septoria-leaf-spot-6.JPG
1/1 [==============================] - 0s 26ms/step
Your leaf disease is --> Tomato___Septoria_leaf_spot


Testing image: tomato-healthy-1.JPG
1/1 [==============================] - 0s 24ms/step
Your leaf disease is --> Tomato___healthy


Testing image: tomato-spider-mites-1.JPG
1/1 [==============================] - 0s 54ms/step
Your leaf disease is --> Tomato___Spider_mites Two-spotted_spider_mite


Testing image: tomato-yellow-curl-3.JPG
1/1 [==============================] - 0s 29ms/step
Your leaf disease is --> Tomato___Tomato_Yellow_Leaf_Curl_Virus


Testing image: tomato-bacterial-spot-4.JPG
1/1 [==============================] - 0s 33ms/step
Your leaf disease is --> Tomato___Bacterial_spot


Testing image: tomato-early-blight-5.JPG
1/1 [==============================] - 0s 23ms/step
Your leaf disease is --> Tomato___Early_blight


Testing image: tomato-early-blight-4.JPG
1/1 [==============================] - 0s 52ms/step
Your leaf